##### Disclaimer: 

- I decided to use Selenium for extracting of information at [Klook](https://www.klook.com/en-PH) since most of leading tourist information website have a dynamic elements that can be tremendous to extract by just using BeautifulSoup alone.

#### In case you miss out to use Selenium.  you can run this code below.

In [ ]:
pip install --upgrade pip

In [ ]:
pip install webdriver-manager

In [ ]:
pip install selenium

In [ ]:
pip install -U selenium

We will now start the drill.

# Extraction of tourist spots

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd

chrome_options = Options()
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.9999.99 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.5",
    "Custom-Header": "Custom Value"
}

for key, value in headers.items():
    chrome_options.add_argument(f'--header={key}:{value}')
#chrome_options.add_argument("--headless")

Sadly,  headless argument can't work with my code.
Its up to you if you want to run all the codes provided below.I ensure all step by step output are showned below are proof that my code works.

### Note: Please do not close the pop up browser. It will automatically close when the extraction of needed information put a finish on.

# Create a defined structured object

In [8]:
def scrape_tourist_spots(URL):
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(URL)
    driver.implicitly_wait(20)
    #Scroll the whole content of website to extract all elements upto the last part of HTML.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait for a few seconds
    time.sleep(5)
    
    #Extract Tourist Spot
    tourist_spot = driver.find_element(By.XPATH,('.//*[@id="activity_title"]/h1/span'))
    tourist_spot = tourist_spot.text
    
    #Extract the Address
    address = driver.find_element(By.XPATH,('.//*[@id="score_participants"]/div[1]/div[2]/div/div/div/div'))
    address = address.text
    
    #Extract the price
    price_content = driver.find_element(By.XPATH, f'.//*[@id="banner_atlas"]/div[2]/div/div')
    price_content_html = price_content.get_attribute("outerHTML")
    price_html = soup(price_content_html, 'html.parser')
    price = price_html.find_all('span', {'class' : 'price-box salling-price'})
    price = price[0].text
    
    #Extract overall ratings
    overall_ratings = driver.find_element(By.XPATH,('.//*[@id="reviews"]/div[1]/div[1]/b'))
    overall_ratings = overall_ratings.text
    
    #content body tag of reviews
    content = driver.find_element(By.CSS_SELECTOR, '.off-screen')
    content_html = content.get_attribute("outerHTML")
    html = soup(content_html, 'html.parser')
     
    #Extract the usernames
    usernames = html.find_all('div', {'class' : 'review-user-name'})
    list_usernames = []
    for u1 in usernames:
        list_usernames.append(u1.text.strip())
    list_usernames = [f'{i1 + 1}. {item1}' for i1, item1 in enumerate(list_usernames)]
    list_usernames = '\n'.join(list_usernames)
    
    #Extract Reviews
    reviews = html.find_all('p', {'class' : 'content'})
    list_reviews = []
    for rev1 in reviews:
        list_reviews.append(rev1.text.strip())
    list_reviews = [f'{i2 + 1}. {item2}' for i2, item2 in enumerate(list_reviews)]
    list_reviews = '\n'.join(list_reviews)
    
    #Compile list into one DataFrame
    Compile = {
        'Name of Tourist Spot / Package' : [tourist_spot],
        'Location of Tourist Spot:' : [address],
        'Tourist Entrance Fee (in PHP):' : [price],
        'Tourist Overall Ratings(/5):' : [overall_ratings],
        'UserNames:' : [list_usernames],
        'Tourist Reviews:' : [list_reviews]    
    }
    Compiled_df = pd.DataFrame(Compile)
    driver.quit()
    return Compiled_df

For now on, we are done with defining our own structure that will return a compiled data frame for us to easily compile later on.

### 1. [Art In Island](https://www.klook.com/en-PH/activity/19055-art-in-island-ticket-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=eb14cc4fa1)

In [9]:
URL = 'https://www.klook.com/en-PH/activity/19055-art-in-island-ticket-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=eb14cc4fa1'
tourist1 = scrape_tourist_spots(URL)
tourist1

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Art In Island: The Media Square Ticket in Manila,"Art in Island: The Media Square, 175 15th Aven...",₱ 800,4.8,1. PamelaGrace\n2. Javier\n3. THERESE\n4. Cher...,1. art in island features such amazing interac...


### 2. [Manila Ocean Park](https://www.klook.com/en-PH/activity/9788-ocean-park-ticket-manila/)

In [10]:
URL = 'https://www.klook.com/en-PH/activity/9788-ocean-park-ticket-manila/'
tourist2 = scrape_tourist_spots(URL)
tourist2

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Manila Ocean Park Attraction Pass,Manila Ocean Park,₱ 680,4.7,1. StephanieKate\n2. Maria Carlyn\n3. Babynico...,1. It's very convenient that I've purchased ou...


### 3. [Fort Santiago](https://www.klook.com/en-PH/activity/86878-fort-santiago-ticket-intramuros-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=047652426a)

In [11]:
URL = 'https://www.klook.com/en-PH/activity/86878-fort-santiago-ticket-intramuros-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=047652426a'
tourist3 = scrape_tourist_spots(URL)
tourist3

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Fort Santiago Ticket in Intramuros Manila,"Fort Santiago, Sta. Clara St., Intramuros, Manila",₱ 50,4.8,1. Mark\n2. Klook User\n3. Emerald\n4. Loverly...,"1. Nice, affordable. You can bring your bike i..."


### 4. [Casa Manila Museum](https://www.klook.com/en-PH/activity/86879-casa-manila-museum-ticket-intramuros/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=0fc78866ca)

In [14]:
URL = 'https://www.klook.com/en-PH/activity/86879-casa-manila-museum-ticket-intramuros/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=0fc78866ca'
tourist4 = scrape_tourist_spots(URL)
tourist4

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Casa Manila Museum Ticket in Intramuros,"Casa Manila Museum, Cor. G. Luna & Real St., I...",₱ 50,5.0,1. Klook User\n2. Klook User\n3. Lis\n4. Leif ...,1. instagrammable place. we enjoyed our visit....


### 5. [The Mind Museum](https://www.klook.com/en-PH/activity/7364-the-mind-museum-all-day-pass-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=77832d5877)

In [15]:
URL = 'https://www.klook.com/en-PH/activity/7364-the-mind-museum-all-day-pass-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=77832d5877'
tourist5 = scrape_tourist_spots(URL)
tourist5

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,The Mind Museum Ticket,The Mind Museum,₱ 360,4.7,1. EUNICE CARLA\n2. Klook User\n3. Catherine\n...,"1. The location was good, convenient and easy ..."


### 6. [Whimsical Wonderland and Artboom](https://www.klook.com/en-PH/activity/79634-whimsical-wonderland-ticket-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=148aafb5e1)

In [16]:
URL = 'https://www.klook.com/en-PH/activity/79634-whimsical-wonderland-ticket-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=148aafb5e1'
tourist6 = scrape_tourist_spots(URL)
tourist6

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Whimsical Wonderland and Artboom Admission Ticket,"4th Floor, Eastwood Mall, Quezon City",₱ 399,4.7,1. Maria Christine\n2. Klook User\n3. Ma Jesus...,1. Thankful to experience magic! It was really...


### 7. [SM by the Bay Amusement Park](https://www.klook.com/en-PH/activity/54250-sm-bay-amusement-park-ride-all-you-can-day-pass/?spm=SearchResult.SearchResult_LIST&clickId=34250124c0)

In [17]:
URL = 'https://www.klook.com/en-PH/activity/54250-sm-bay-amusement-park-ride-all-you-can-day-pass/?spm=SearchResult.SearchResult_LIST&clickId=34250124c0'
tourist7 = scrape_tourist_spots(URL)
tourist7

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,SM by the Bay Amusement Park Ride-All-You-Can ...,SM by the Bay Amusement Park,₱ 402,4.5,1. Klook User\n2. Klook User\n3. Lhyne\n4. cap...,1. Easy to redeem. Affordable and very accessi...


### 8. [The Dessert Museum](https://www.klook.com/en-PH/activity/9526-dessert-museum-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=b61929314e)

In [18]:
URL = 'https://www.klook.com/en-PH/activity/9526-dessert-museum-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=b61929314e'
tourist8 = scrape_tourist_spots(URL)
tourist8

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,The Dessert Museum Ticket in Manila,The Dessert Museum,₱ 699,4.6,1. Norman\n2. Luibelle\n3. RonnaMae\n4. APRIL ...,1. The location was good and convenient. Excel...


### 9. [PLAY at Okada Manila](https://www.klook.com/en-PH/activity/84288-play-admission-ticket-okada-manila/?spm=Activity.YouMayAlsoLike%3Acommon%3A%3Arecommend_activity%3AActivity_LIST&clickId=006fe32111)

In [19]:
URL = 'https://www.klook.com/en-PH/activity/84288-play-admission-ticket-okada-manila/?spm=Activity.YouMayAlsoLike%3Acommon%3A%3Arecommend_activity%3AActivity_LIST&clickId=006fe32111'
tourist9 = scrape_tourist_spots(URL)
tourist9

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,PLAY at Okada Manila Admission Ticket,PLAY at Okada Manila,₱ 588,4.8,1. CRISTINASWEN\n2. Marcela Lean\n3. karen\n4....,"1. Clean, tidy and smells clean. Safe for kids..."


### 10. [Nightmares Manila](https://www.klook.com/en-PH/activity/47533-nightmares-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=478c5bd764)

In [20]:
URL = 'https://www.klook.com/en-PH/activity/47533-nightmares-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=478c5bd764'
tourist10 = scrape_tourist_spots(URL)
tourist10

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Nightmares Manila Ticket,"Dr. A. Santos Ave. cor. Rainbow Drive, Goodwil...","₱ 2,000",4.8,1. YEN\n2. MARY JEAN\n3. MARIE\n4. Eloisa\n5. ...,1. We want to go back! 😁\n\nExtraordinary expe...


### 11. [Baluarte de San Diego](https://www.klook.com/en-PH/activity/86882-baluarte-de-san-diego-ticket-intramuros-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=35af08580a)

In [21]:
URL = 'https://www.klook.com/en-PH/activity/86882-baluarte-de-san-diego-ticket-intramuros-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=35af08580a'
tourist11 = scrape_tourist_spots(URL)
tourist11

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Baluarte de San Diego Ticket in Intramuros Manila,"Baluarte de San Diego, Sta. Lucia St., Intramu...",₱ 50,4.7,1. Klook User\n2. Loverly\n3. MariaIsabel\n4. ...,1. instagrammable place. we enjoyed our visit....


### 12. [Museo de Intramuros](https://www.klook.com/en-PH/activity/86885-museo-de-intramuros-ticket-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=e3443e9993)

In [22]:
URL = 'https://www.klook.com/en-PH/activity/86885-museo-de-intramuros-ticket-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=e3443e9993'
tourist12 = scrape_tourist_spots(URL)
tourist12

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Museo de Intramuros Ticket in Manila,"Museo de Intramuros, cor. Arzobispo, Anda St.,...",₱ 50,4.7,1. Klook User\n2. Harold Jay\n3. 晨奕,1. instagrammable place. we enjoyed our visit....


### 13. [Paradizoo](https://www.klook.com/en-PH/activity/13424-paradizoo-admission-ticket-tagaytay-manila/?spm=City.Popular%3Aany%3A%3AHotAct%3ACard_LIST&clickId=8f2980e827)

In [24]:
URL = 'https://www.klook.com/en-PH/activity/13424-paradizoo-admission-ticket-tagaytay-manila/?spm=City.Popular%3Aany%3A%3AHotAct%3ACard_LIST&clickId=8f2980e827'
tourist13 = scrape_tourist_spots(URL)
tourist13

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Paradizoo Ticket in Tagaytay,Paradizoo,₱ 159,4.2,1. Thea\n2. Talatayod\n3. Thea\n4. jade\n5. Ra...,1. it was a great experience...i like how it h...


### 14. [Sky Ranch Tagaytay](https://www.klook.com/en-PH/activity/14539-sky-ranch-tagaytay-ride-all-you-can-day-pass-manila/?spm=City.Popular%3Aany%3A%3AHotAct%3ACard_LIST&clickId=aa83275e6e)

In [25]:
URL = 'https://www.klook.com/en-PH/activity/14539-sky-ranch-tagaytay-ride-all-you-can-day-pass-manila/?spm=City.Popular%3Aany%3A%3AHotAct%3ACard_LIST&clickId=aa83275e6e'
tourist14 = scrape_tourist_spots(URL)
tourist14

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Sky Ranch Tagaytay Ride-All-You-Can Day Pass,"Sky Ranch Tagaytay, Tagaytay-Nasugbu Highway, ...",₱ 388,4.6,1. Jennyfer\n2. Bernadette\n3. Joan Rae\n4. Kl...,1. Super convenient and hassle free booking. W...


### 15. [Zoori at Residence Inn](https://www.klook.com/en-PH/activity/13421-zoori-residence-admission-ticket-tagaytay-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=4be59eb26f)

In [26]:
URL = 'https://www.klook.com/en-PH/activity/13421-zoori-residence-admission-ticket-tagaytay-manila/?spm=City.ActivityList%3Aany%3A%3ACityActivitiesFilter%3ACard_LIST&clickId=4be59eb26f'
tourist15 = scrape_tourist_spots(URL)
tourist15

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Zoori at Residence Inn Ticket in Tagaytay,Residence Inn Zoo,₱ 159,4.2,1. Klook User\n2. Aaron Ryan\n3. Aaron\n4. Kar...,1. Easy to use app and legit naman tlga\n acce...


### 16. [Enchanted Kingdom](https://www.klook.com/en-PH/activity/7350-enchanted-kingdom-laguna-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=1fc47650dc)

In [27]:
URL = 'https://www.klook.com/en-PH/activity/7350-enchanted-kingdom-laguna-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=1fc47650dc'
tourist16 = scrape_tourist_spots(URL)
tourist16

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Enchanted Kingdom Ticket in Laguna,Enchanted Kingdom,₱ 400,4.8,1. ktrn\n2. Janne Erika\n3. Christine Joy\n4. ...,1. It was raining but the park was still packe...


### 17. [Dinosaurs Island](https://www.klook.com/en-PH/activity/14252-dinosaurs-island-admission-ticket-clark/?spm=Attraction_ActivityList.Activity_LIST&clickId=1dfe3803c8)

In [28]:
URL = 'https://www.klook.com/en-PH/activity/14252-dinosaurs-island-admission-ticket-clark/?spm=Attraction_ActivityList.Activity_LIST&clickId=1dfe3803c8'
tourist17 = scrape_tourist_spots(URL)
tourist17

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Dinosaurs Island Ticket in Clark,"Dinosaurs Island, Clark Picnic Grounds, Gil Pu...",₱ 699,4.8,1. Klook User\n2. Michele\n3. Ma. Karina Saman...,1. Tip: purchase your tickets here in Klook! I...


### 18. [Tales of Illumina](https://www.klook.com/en-PH/activity/66149-tales-illumina-ticket-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=a48d163b69)

In [29]:
URL = 'https://www.klook.com/en-PH/activity/66149-tales-illumina-ticket-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=a48d163b69'
tourist18 = scrape_tourist_spots(URL)
tourist18

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Tales of Illumina Ticket in Manila,Tales of Illumina,₱ 649,4.7,1. Klook User\n2. Kate\n3. VanessaJoyce\n4. Pr...,"1. It was a great experience, this place is an..."


### 19. [Splash Island](https://www.klook.com/en-PH/activity/10662-splash-island-ticket-laguna-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=a54ab13045)

In [30]:
URL = 'https://www.klook.com/en-PH/activity/10662-splash-island-ticket-laguna-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=a54ab13045'
tourist19 = scrape_tourist_spots(URL)
tourist19

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Splash Island Ticket in Laguna,"Splash Island, Southwoods Ecocentrum Binan, La...",₱ 499,4.5,1. Regie\n2. Brian\n3. Steven polo\n4. Michell...,1. A great experience for your family and to h...


### 20. [Inflatable Island Beach Club](https://www.klook.com/en-PH/activity/8402-inflatable-island-subic-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=d663a14859)

In [31]:
URL = 'https://www.klook.com/en-PH/activity/8402-inflatable-island-subic-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=d663a14859'
tourist20 = scrape_tourist_spots(URL)
tourist20

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Inflatable Island Beach Club Ticket in Subic,Inflatable Island,₱ 899,4.5,1. Kashmere\n2. Mary Joy Grace\n3. Kalmia Mimo...,1. The playtime is enjoyable yet it made us ve...


### 21. [Adventure Beach Waterpark](https://www.klook.com/en-PH/activity/11504-adventure-beach-waterpark-admission-ticket-subic-bay-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=58b49036bb)

In [32]:
URL = 'https://www.klook.com/en-PH/activity/11504-adventure-beach-waterpark-admission-ticket-subic-bay-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=58b49036bb'
tourist21 = scrape_tourist_spots(URL)
tourist21

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Adventure Beach Waterpark Ticket in Subic,Adventure Waterpark,₱ 528,4.6,1. Angelyn\n2. Jurist Castrence\n3. Janette\n4...,"1. Nice place , so clean that's why, we felt s..."


### 22. [Avilon Zoo](https://www.klook.com/en-PH/activity/22280-avilon-zoo-ticket-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=5bdf87352a)

In [33]:
URL = 'https://www.klook.com/en-PH/activity/22280-avilon-zoo-ticket-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=5bdf87352a'
tourist22 = scrape_tourist_spots(URL)
tourist22

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Avilon Zoo Ticket in Rizal,Avilon Zoo,₱ 600,4.9,1. Klook User\n2. PAULINE\n3. Judie Anne\n4. j...,1. Great weather condition. Nice place to visi...


### 23. [Ocean Adventure](https://www.klook.com/en-PH/activity/11498-ocean-adventure-admission-ticket-subic-bay-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=1e2c40befa)

In [34]:
URL = 'https://www.klook.com/en-PH/activity/11498-ocean-adventure-admission-ticket-subic-bay-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=1e2c40befa'
tourist23 = scrape_tourist_spots(URL)
tourist23

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Ocean Adventure Ticket in Subic,Ocean Adventure,₱ 680,4.7,1. Georgina\n2. Rabin\n3. Klook User\n4. LENNE...,1. It was an amazing experience and very worth...


### 24. [Zoocobia Fun Zoo](https://www.klook.com/en-PH/activity/13443-zoocobia-admission-ticket-clark-pampanga-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=0141145598)

In [35]:
URL = 'https://www.klook.com/en-PH/activity/13443-zoocobia-admission-ticket-clark-pampanga-manila/?spm=Attraction_ActivityList.Activity_LIST&clickId=0141145598'
tourist24 = scrape_tourist_spots(URL)
tourist24

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Zoocobia Fun Zoo Ticket in Clark,Zoocobia,₱ 236,4.3,1. Glizzette\n2. Jazmin Joy\n3. Maria Ronissa ...,1. the kids had so much fun!!! we gonna book a...


### 25. [Clark Safari and Adventure](https://www.klook.com/en-PH/activity/73222-clark-safari-adventure-park-ticket/?spm=Attraction_ActivityList.Activity_LIST&clickId=5eb2b2faf8)

In [36]:
URL = 'https://www.klook.com/en-PH/activity/73222-clark-safari-adventure-park-ticket/?spm=Attraction_ActivityList.Activity_LIST&clickId=5eb2b2faf8'
tourist25 = scrape_tourist_spots(URL)
tourist25

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Clark Safari and Adventure Park Ticket,"67 Jose Abad Santos Ave, Clark Freeport, Mabal...",₱ 599,4.6,1. Raezelle\n2. lorelyn\n3. miriam\n4. Ma Kris...,1. Purchasing tickets from Klook was hassle fr...


# FINALLY

In [37]:
tourist_sites = [globals()[f"tourist{i}"] for i in range(1, 26)]
Merge_df = pd.concat(tourist_sites)
Merge_df.to_csv('Tourist_site.csv', index = False) #save to csv file.
Tourist_Site_df = pd.read_csv('Tourist_site.csv')
Tourist_Site_df

,Name of Tourist Spot / Package,Location of Tourist Spot:,Tourist Entrance Fee (in PHP):,Tourist Overall Ratings(/5):,UserNames:,Tourist Reviews:
0,Art In Island: The Media Square Ticket in Manila,"Art in Island: The Media Square, 175 15th Aven...",₱ 800,4.8,1. PamelaGrace\n2. Javier\n3. THERESE\n4. Cher...,1. art in island features such amazing interac...
1,Manila Ocean Park Attraction Pass,Manila Ocean Park,₱ 680,4.7,1. StephanieKate\n2. Maria Carlyn\n3. Babynico...,1. It's very convenient that I've purchased ou...
2,Fort Santiago Ticket in Intramuros Manila,"Fort Santiago, Sta. Clara St., Intramuros, Manila",₱ 50,4.8,1. Mark\n2. Klook User\n3. Emerald\n4. Loverly...,"1. Nice, affordable. You can bring your bike i..."
3,Casa Manila Museum Ticket in Intramuros,"Casa Manila Museum, Cor. G. Luna & Real St., I...",₱ 50,5.0,1. Klook User\n2. Klook User\n3. Lis\n4. Leif ...,1. instagrammable place. we enjoyed our visit....
4,The Mind Museum Ticket,The Mind Museum,₱ 360,4.7,1. EUNICE CARLA\n2. Klook User\n3. Catherine\n...,"1. The location was good, convenient and easy ..."
5,Whimsical Wonderland and Artboom Admission Ticket,"4th Floor, Eastwood Mall, Quezon City",₱ 399,4.7,1. Maria Christine\n2. Klook User\n3. Ma Jesus...,1. Thankful to experience magic! It was really...
6,SM by the Bay Amusement Park Ride-All-You-Can ...,SM by the Bay Amusement Park,₱ 402,4.5,1. Klook User\n2. Klook User\n3. Lhyne\n4. cap...,1. Easy to redeem. Affordable and very accessi...
7,The Dessert Museum Ticket in Manila,The Dessert Museum,₱ 699,4.6,1. Norman\n2. Luibelle\n3. RonnaMae\n4. APRIL ...,1. The location was good and convenient. Excel...
8,PLAY at Okada Manila Admission Ticket,PLAY at Okada Manila,₱ 588,4.8,1. CRISTINASWEN\n2. Marcela Lean\n3. karen\n4....,"1. Clean, tidy and smells clean. Safe for kids..."
9,Nightmares Manila Ticket,"Dr. A. Santos Ave. cor. Rainbow Drive, Goodwil...","₱ 2,000",4.8,1. YEN\n2. MARY JEAN\n3. MARIE\n4. Eloisa\n5. ...,1. We want to go back! 😁\n\nExtraordinary expe...
